In [1]:
import tensorflow as tf
from tensorflow.python import keras #for some reason PyCharm doesn't detect tensorflow.keras (idk why)
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input

vgg = VGG19(include_top=False) #we only need the convolutional layers

In [3]:
def load_img(path):
    max_dim = 512
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    
    new_shape = tf.cast(tf.shape(img)[:-1] * max_dim/max(tf.shape(img)), dtype=tf.int32)#rescale image to 512
    img = tf.image.resize(img, new_shape)
    
    
    return img[tf.newaxis, :] #vgg requires batched input

def imshow(image, title=None):
    if len(tf.shape(image)) > 3:
        image = tf.squeeze(image, axis=0)
    plt.imshow(image)
    if title:
        plt.title(title)
     

In [4]:
def layer_models(layers):
    vgg = VGG19(include_top=False)
    vgg.trainable = False
    
    outputs = [vgg.get_layer(layer).output for layer in layers]
    return tf.keras.Model(vgg.inputs, outputs)

In [12]:
def gram_matrix(tensor):
    temp = tf.linalg.einsum('ijka, ijkb->iab', tensor, tensor)
    return temp/tf.cast(tf.shape(tensor)[1] * tf.shape(tensor)[2], tf.float32) #loss is relative to 

In [13]:
class StyleTransfer(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleTransfer, self).__init__()
        self.vgg = layer_models(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False
    def call(self, inputs):
        inputs = inputs*255.0
        preprocessed_inputs = preprocess_input(inputs)
        outputs = self.vgg(preprocessed_inputs)
        style_outputs = outputs[:self.num_style_layers]
        content_outputs = outputs[self.num_style_layers:]
        
        style_outputs = [gram_matrix(style) for style in style_outputs]
        
        style_dictionary = {style : output for style, output in zip(self.style_layers, style_outputs)}
        content_dictionary = {content : output for content, output in zip(self.content_layers, content_outputs)}
        
        return {"style" : style_dictionary, "content" : content_dictionary}

In [15]:
#Identify style and content layers
style_layers = ["block1_pool", "block2_pool", "block3_pool", "block4_pool"]
content_layers = ["block5_pool"]

#Content and style images
content_image = load_img("nighthawks.jpg")
style_image = load_img("starryNightResized.jpg")

extractor = StyleTransfer(style_layers, content_layers)

extractor(tf.constant(content_image))

{'style': {'block1_pool': <tf.Tensor: shape=(1, 64, 64), dtype=float32, numpy=
  array([[[  9008.886 ,   9052.644 ,  14108.863 , ...,   4225.556 ,
             4276.1846,   5502.0405],
          [  9052.644 ,  17271.033 ,  38150.99  , ...,   6908.8457,
             5921.324 ,   4853.137 ],
          [ 14108.863 ,  38150.99  , 140497.86  , ...,   4234.1377,
            13382.983 ,   6955.66  ],
          ...,
          [  4225.556 ,   6908.8457,   4234.1377, ...,  10605.417 ,
             4219.266 ,   2878.243 ],
          [  4276.1846,   5921.324 ,  13382.983 , ...,   4219.266 ,
             9844.253 ,   4172.7837],
          [  5502.0405,   4853.137 ,   6955.66  , ...,   2878.243 ,
             4172.7837,   8622.902 ]]], dtype=float32)>,
  'block2_pool': <tf.Tensor: shape=(1, 128, 128), dtype=float32, numpy=
  array([[[ 64092.934,  15731.841,  12962.774, ...,  43833.605,
            37527.773,  11433.6  ],
          [ 15731.841, 101774.42 ,  13981.542, ...,  32397.666,
            154